In [1]:
import glob

In [2]:
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Google Cloud Speech API samplbbbe application using the streaming API.

NOTE: This module requires the additional dependency `pyaudio`. To install
using pip:

    pip install pyaudio

Example usage:
    python transcribe_streaming_mic.py
"""

# [START speech_transcribe_streaming_mic]
from __future__ import division

import re
import sys

from google.cloud import speech

import pyaudio
from six.moves import queue
import time
import glob

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms


class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""

    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            # The API currently only supports 1-channel (mono) audio
            # https://goo.gl/z757pE
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            # Run the audio stream asynchronously to fill the buffer object.
            # This is necessary so that the input device's buffer doesn't
            # overflow while the calling thread makes network requests, etc.
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        # Signal the generator to terminate so that the client's
        # streaming_recognize method will not block the process termination.
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            # Use a blocking get() to ensure there's at least one chunk of
            # data, and stop iteration if the chunk is None, indicating the
            # end of the audio stream.
            chunk = self._buff.get()
            if chunk is None:
                return
            data = [chunk]

            # Now consume whatever other data's still buffered.
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b"".join(data)


def listen_print_loop(responses,user_id):
    """Iterates through server responses and prints them.

    The responses passed is a generator that will block until a response
    is provided by the server.

    Each response may contain multiple results, and each result may contain
    multiple alternatives; for details, see https://goo.gl/tjCPAU.  Here we
    print only the transcription for the top alternative of the top result.

    In this case, responses are provided for interim results as well. If the
    response is an interim one, print a line feed at the end of it, to allow
    the next result to overwrite it, until the response is a final one. For the
    final one, print a newline to preserve the finalized transcription.
    """
    t = str(int(time.time()))
    num_chars_printed = 0
    for response in responses:
        if not response.results:
            continue

        # The `results` list is consecutive. For streaming, we only care about
        # the first result being considered, since once it's `is_final`, it
        # moves on to considering the next utterance.
        result = response.results[0]
        if not result.alternatives:
            continue

        # Display the transcription of the top alternative.
        transcript = result.alternatives[0].transcript

        # Display interim results, but with a carriage return at the end of the
        # line, so subsequent lines will overwrite them.
        #
        # If the previous result was longer than this one, we need to print
        # some extra spaces to overwrite the previous result
        overwrite_chars = " " * (num_chars_printed - len(transcript))

        if not result.is_final:
                
            sys.stdout.write(transcript + overwrite_chars + "\r")
            sys.stdout.flush()

            num_chars_printed = len(transcript)

        else:
            print(transcript + overwrite_chars)
            
            with open("./"+t+"_"+user_id+"_my.txt","a") as f :
                f.writelines(transcript + overwrite_chars+"\n")
                
            # Exit recognition if any of the transcribed phrases could be
            # one of our keywords.
            if re.search(r"\b(exit|quit)\b", transcript, re.I):
                print("Exiting..")
                break

            num_chars_printed = 0


def main(user_id):
    # See http://g.co/cloud/speech/docs/languages
    # for a list of supported languages.
    language_code = "ko-KR"  # a BCP-47 language tag

    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code
    )

    streaming_config = speech.StreamingRecognitionConfig(
        config=config, interim_results=True
    )

    global stream
    with MicrophoneStream(RATE, CHUNK) as stream:
        #print("step1")
        audio_generator = stream.generator()
        
        requests = (
            speech.StreamingRecognizeRequest(audio_content=content)
            for content in audio_generator
        )
        #print("step2")
        responses = client.streaming_recognize(streaming_config, requests)
        #print("step3")
        print(stream.closed)
        # stream.closed = True
        print(stream.closed)
        # Now, put the transcription responses to use.
        listen_print_loop(responses,user_id)
        #print("step4")
        # stream.closed = True

# if __name__ == "__main__":
#     main()
# [END speech_transcribe_streaming_mic]


In [3]:
!pip install konlpy

In [4]:
!pip install -U flask-cors

Requirement already up-to-date: flask-cors in c:\programdata\anaconda3\lib\site-packages (3.0.10)


In [5]:
from flask import Flask,redirect,request,Response,send_file
from flask_cors import CORS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
import csv

In [ ]:
okt = Okt()

app = Flask(__name__)
CORS(app)

@app.route('/user')
def user():
    user_id = request.args['id']
    main(user_id)
    # 이부분이 중단되었을때 코드를 작성하면 되겠죠
    
    return 'Hello, User!'

@app.route('/end')
def end():
    stream.closed = True
    return 'end'
@app.route('/txthabit')
def txt_habit():
    print('step1')
    
    user_id = request.args['id']
    
    output = glob.glob('./*'+user_id+'_my.txt')
    output = sorted(output)
    
    f1 = open(output[-1],'r',encoding='euc-kr')
    txt = f1.readlines()
    f1.close()
    print(txt)
    morph_list = []
    for i in txt :
        morphs = okt.morphs(i)
        for morph in morphs:
            morph_list.append(morph)
        
    def counter(morph_list):
        word_count = {}
        for word in morph_list:
            if word in word_count :
                word_count[word] += 1
            else :
                word_count[word] = 1
        return word_count

    word_count = counter(morph_list)
    
    # 정렬
    word_count = sorted(word_count.items(), key = lambda x:x[1], reverse=True)
    fields = ['습관어' , '횟수']
    rows = word_count
    
    with open(r'test1.csv', 'w', newline='\n') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(rows)
        
    test_all_word = pd.read_csv('./test1.csv', encoding='euc-kr')
    
    data1 = morph_list
    cnt = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    morph_list

    for item in data1:
        if item =='아':
            cnt[0] = cnt[0] +1
        elif item == '음':
            cnt[1] = cnt[1] +1
        elif item == '그':
            cnt[2] = cnt[2] +1
        elif item == '어':  
            cnt[3] = cnt[3]+1
        elif item == '그니까':
            cnt[4] = cnt[4] +1
        elif item == '저':
            cnt[5] = cnt[5] +1
        elif item == '이제':
            cnt[6] = cnt[6] +1
        elif item == '막':
            cnt[7] = cnt[7] +1
        elif item == '에':
            cnt[8] = cnt[8] +1
            
    # 습관어 list
    Habitual_word = ['아','음','그','어','그니까','저','이제','막','에']
    
    # 최근 회차 text 결과
    Habitual_word_cnt = cnt
    display(Habitual_word_cnt)
    x = np.arange(len(Habitual_word))
    
    plt.figure(figsize=(10,5))
    plt.rc('font', family='Malgun Gothic')
    plt.xlim(-1,9)
    plt.ylim(0,15)
    plt.bar(x-0.1, Habitual_word_cnt, width =0.3, color ='yellow',align='center')
    plt.title('습관어 빈도수 분석결과', fontsize = 20)
    plt.xlabel('<습관어 종류>', fontsize = 14)
    plt.ylabel('<빈도수>', fontsize = 14)
    plt.rc('xtick', labelsize = 12)
    plt.rc('ytick', labelsize = 12)
    #plt.rcParams['figure.figuresize'] = (10,4)
    plt.grid(axis='y', linestyle='--', alpha = 0.5)
    plt.xticks(x, Habitual_word, fontsize=8)

    for i, v in enumerate(x):
        aa = "{:,}회    ".format(Habitual_word_cnt[i])
        plt.text(v,Habitual_word_cnt[i],aa,fontsize=9,color='#997000',
                  horizontalalignment='center',verticalalignment='bottom')  


    plt.savefig('./습관어의 종류와 빈도수_최근회차.png',bbox_inches='tight')
    
    
    return send_file('./습관어의 종류와 빈도수_최근회차.png')
    
# -------------------- 습관어 빈도수 최근회차 차트 -----------------------------------
    
@app.route('/txtchbi')
def txt_chbi():
    print('step1')
    
    user_id = request.args['id']
    
    output = glob.glob('./*'+user_id+'_my.txt')
    output = sorted(output)
    
    f1 = open(output[-1],'r',encoding='euc-kr')
    txt = f1.readlines()
    f1.close()
    print(txt)
    morph_list = []
    for i in txt :
        morphs = okt.morphs(i)
        for morph in morphs:
            morph_list.append(morph)
        
    def counter(morph_list):
        word_count = {}
        for word in morph_list:
            if word in word_count :
                word_count[word] += 1
            else :
                word_count[word] = 1
        return word_count

    word_count = counter(morph_list)
    
    # 정렬
    word_count = sorted(word_count.items(), key = lambda x:x[1], reverse=True)
    fields = ['습관어' , '횟수']
    rows = word_count
    
    with open(r'test1.csv', 'w', newline='\n') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(rows)
        
    test_all_word = pd.read_csv('./test1.csv', encoding='euc-kr')
    
    data1 = morph_list
    cnt = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    morph_list

    for item in data1:
        if item =='아':
            cnt[0] = cnt[0] +1
        elif item == '음':
            cnt[1] = cnt[1] +1
        elif item == '그':
            cnt[2] = cnt[2] +1
        elif item == '어':  
            cnt[3] = cnt[3]+1
        elif item == '그니까':
            cnt[4] = cnt[4] +1
        elif item == '저':
            cnt[5] = cnt[5] +1
        elif item == '이제':
            cnt[6] = cnt[6] +1
        elif item == '막':
            cnt[7] = cnt[7] +1
        elif item == '에':
            cnt[8] = cnt[8] +1
            
    # 습관어 list
    Habitual_word = ['아','음','그','어','그니까','저','이제','막','에']
    
    # 최근 회차 text 결과
    Habitual_word_cnt = cnt
    display(Habitual_word_cnt)
    x = np.arange(len(Habitual_word))
    
    
    f2 = open(output[-2],'r',encoding='euc-kr')
    txt2 = f2.readlines()
    f2.close()
    
    morph_list2 = []
    for i in txt2 :
        morphs = okt.morphs(i)
        for morph in morphs:
            morph_list2.append(morph)
    
    
    def counter(morph_list2):
        word_count2 = {}
        for word in morph_list2:
            if word in word_count2 :
                word_count2[word] += 1
            else :
                word_count2[word] = 1
        return word_count2

    word_count2 = counter(morph_list2)
    
    # 정렬
    word_count2 = sorted(word_count2.items(), key = lambda x:x[1], reverse=True)
    
    fields = ['습관어' , '횟수']
    rows = word_count2
    
    with open(r'test2.csv', 'w', newline='\n') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(rows)
        
    test_all_word2 = pd.read_csv('./test2.csv', encoding='euc-kr')
    
    data2 = morph_list2
    cnt2 = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    
    for item in data2:
        if item =='아':
            cnt2[0] = cnt2[0] +1
        elif item == '음':
            cnt2[1] = cnt2[1] +1
        elif item == '그':
            cnt2[2] = cnt2[2] +1
        elif item == '어':  
            cnt2[3] = cnt2[3]+1
        elif item == '그니까':
            cnt2[4] = cnt2[4] +1
        elif item == '저':
            cnt2[5] = cnt2[5] +1
        elif item == '이제':
            cnt2[6] = cnt2[6] +1
        elif item == '막':
            cnt2[7] = cnt2[7] +1
        elif item == '에':
            cnt2[8] = cnt2[8] +1
    
    # 습관어 list
    Habitual_word = ['아','음','그','어','그니까','저','이제','막','에']

    # 최근 회차 text 결과
    Habitual_word_cnt2 = cnt2
    display(Habitual_word_cnt2)
    x = np.arange(len(Habitual_word))
    
    
    plt.figure(figsize=(10,5))
    plt.rc('font', family='Malgun Gothic')
    plt.xlim(-1,9)
    plt.ylim(0,15)
    p1 = plt.bar(x-0.1, Habitual_word_cnt, width =0.3, color ='yellow',align='center')
    #Habitual_word_cnt2 = [2,3,4,2,5,4,2,3,5]
    p2 = plt.bar(x+0.2, Habitual_word_cnt2, width =0.3, color ='gray',align='center')
    plt.legend((p1[0],p2[0]),('현재결과','직전결과'))

    plt.title('습관어 빈도수 분석결과', fontsize = 20)
    plt.xlabel('<습관어 종류>', fontsize = 14)
    plt.ylabel('<빈도수>', fontsize = 14)
    plt.rc('xtick', labelsize = 12)
    plt.rc('ytick', labelsize = 12)
    #plt.rcParams['figure.figuresize'] = (10,4)
    plt.grid(axis='y', linestyle='--', alpha = 0.5)
    plt.xticks(x, Habitual_word, fontsize=8)

    for i, v in enumerate(x):
        aa = "{:,}회    ".format(Habitual_word_cnt[i])
        plt.text(v,Habitual_word_cnt[i],aa,fontsize=9,color='#997000',
                  horizontalalignment='center',verticalalignment='bottom')

    for i, v in enumerate(x):
        aa = "       {:,}회".format(Habitual_word_cnt2[i])
        plt.text(v,Habitual_word_cnt2[i],aa,fontsize=9,color='black',
                  horizontalalignment='center',verticalalignment='bottom')   


    plt.savefig('./습관어의 종류와 빈도수_비교.png',bbox_inches='tight')
    
    
    return send_file('./습관어의 종류와 빈도수_비교.png')
# -------------------- 빈도수 비교 차트 -----------------------------------


@app.route('/txtpos')
def txt_pos():
    print('step1')
    
    user_id = request.args['id']
    
    output = glob.glob('./*'+user_id+'_my.txt')
    output = sorted(output)
    
    f1 = open(output[-1],'r',encoding='euc-kr')
    txt = f1.read()
    f1.close()
    print(txt)
    
    rs = okt.pos(txt)
    df = pd.DataFrame(rs)
    Noun_df = df[(df[1] == 'Noun')]
    Noun_df.drop(1, axis = 1, inplace=True)
    pos_list=np.array(Noun_df[0].tolist())
    
    def counter(pos_list):
        pos_count = {}
        for word in pos_list:
            if word in pos_count :
                pos_count[word] += 1
            else :
                pos_count[word] = 1
    return pos_count
    

    pos_count = counter(pos_list)
    
    # 정렬
    pos_count = sorted(pos_count.items(), key = lambda x:x[1], reverse=True)
    fields = ['단어' , '횟수']
    rows = pos_coun
    
    with open(r'pos_test.csv', 'w', newline='\n') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(rows)
        
    test_pos_word = pd.read_csv('./pos_test1.csv', encoding='euc-kr')
    
    
    pos_data1 = test_pos_word[['단어']]
    pos_data2 = test_pos_word[['횟수']]
    
    posword = test_pos_word['단어'].dropna().values.tolist()[:6]
    posword_count = test_pos_word['횟수'].values.tolist()[:6]
    x = np.arange(len(posword))

    plt.figure(figsize=(10,5))
    plt.rc('font', family='Malgun Gothic')
    plt.xlim(-1,6)
    plt.ylim(0,15)
    plt.bar(posword, posword_count, width =0.5, color ='yellow',align='center')
    plt.title('단어 빈도수 분석결과', fontsize = 20)
    plt.xlabel('<단어 종류>', fontsize = 14)
    plt.ylabel('<빈도수>', fontsize = 14)
    plt.rc('xtick', labelsize = 12)
    plt.rc('ytick', labelsize = 12)
    #plt.rcParams['figure.figuresize'] = (10,4)
    plt.grid(axis='y', linestyle='--', alpha = 0.5)

    for i, v in enumerate(x):
        aa = "{:,}회".format(posword_count[i])
        plt.text(v,posword_count[i],aa,fontsize=9,color='#000000',
                  horizontalalignment='center',verticalalignment='bottom')


    plt.savefig('./단어의 종류와 빈도수.png',bbox_inches='tight')
    
    
    return send_file('./단어의 종류와 빈도수.png')
# -------------------- 빈도수 최빈명사 차트 -----------------------------------


if __name__ == '__main__':
    app.run(host="localhost",port="9001")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9001/ (Press CTRL+C to quit)


False
False
저희는 스마트인재개발원에서 자바기반 웹개발 가정을 교육받은 교육생 드립니다 저희 교육과정에는 막 자바프로그래밍 데이터베이스 실습 파이썬을 활용한여 데이터 분석 및 지각하던 있었으며 이제 데이터 활용하여 자바기반의 웹 서비스에 파이썬을 통한 데이터 분석을 융합하여 또박또박 일하는 STP 분석 웹서비스를 프로젝트로 개발하게 되었습니다
지금 이거 들어간 거예요
 그거는 따로 녹음 파일로 갖고 이거 보고 이거 소리도 안 들어


[2021-02-25 17:53:05,186] ERROR in app: Exception on /user [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\google\api_core\grpc_helpers.py", line 113, in next
    return six.next(self._wrapped)
  File "C:\ProgramData\Anaconda3\lib\site-packages\grpc\_channel.py", line 416, in __next__
    return self._next()
  File "C:\ProgramData\Anaconda3\lib\site-packages\grpc\_channel.py", line 803, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.OUT_OF_RANGE
	details = "Audio Timeout Error: Long duration elapsed without audio. Audio should be sent close to real time."
	debug_error_string = "{"created":"@1614243185.185000000","description":"Error received from peer ipv4:216.58.197.170:443","file":"src/core/lib/surface/call.cc","file_line":1068,"grpc_message":"Audio Timeout Error: Long duration elapsed without audio. Audio should be sent close to real time.","grpc_s

 이거 보고 이거 보고 집에서 핸드폰으로 보면 되잖아 지금 카톡으로 보내 드릴게요 좀 보내지 않을 거야
step1
step1
step1
['저희는 스마트인재개발원에서 자바기반 웹개발 가정을 교육받은 교육생 드립니다 저희 교육과정에는 막 자바프로그래밍 데이터베이스 실습 파이썬을 활용한여 데이터 분석 및 지각하던 있었으며 이제 데이터 활용하여 자바기반의 웹 서비스에 파이썬을 통한 데이터 분석을 융합하여 또박또박 일하는 STP 분석 웹서비스를 프로젝트로 개발하게 되었습니다\n', '지금 이거 들어간 거예요\n', ' 그거는 따로 녹음 파일로 갖고 이거 보고 이거 소리도 안 들어\n', ' 이거 보고 이거 보고 집에서 핸드폰으로 보면 되잖아 지금 카톡으로 보내 드릴게요 좀 보내지 않을 거야\n']
저희는 스마트인재개발원에서 자바기반 웹개발 가정을 교육받은 교육생 드립니다 저희 교육과정에는 막 자바프로그래밍 데이터베이스 실습 파이썬을 활용한여 데이터 분석 및 지각하던 있었으며 이제 데이터 활용하여 자바기반의 웹 서비스에 파이썬을 통한 데이터 분석을 융합하여 또박또박 일하는 STP 분석 웹서비스를 프로젝트로 개발하게 되었습니다
지금 이거 들어간 거예요
 그거는 따로 녹음 파일로 갖고 이거 보고 이거 소리도 안 들어
 이거 보고 이거 보고 집에서 핸드폰으로 보면 되잖아 지금 카톡으로 보내 드릴게요 좀 보내지 않을 거야

['저희는 스마트인재개발원에서 자바기반 웹개발 가정을 교육받은 교육생 드립니다 저희 교육과정에는 막 자바프로그래밍 데이터베이스 실습 파이썬을 활용한여 데이터 분석 및 지각하던 있었으며 이제 데이터 활용하여 자바기반의 웹 서비스에 파이썬을 통한 데이터 분석을 융합하여 또박또박 일하는 STP 분석 웹서비스를 프로젝트로 개발하게 되었습니다\n', '지금 이거 들어간 거예요\n', ' 그거는 따로 녹음 파일로 갖고 이거 보고 이거 소리도 안 들어\n', ' 이거 보고 이거 보고 집에서 핸드폰으로 보면 되잖아 지금 카톡으로 보내 드릴게요 좀 보내지 않을 거야\n']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
[2021-02-25 18:18:26,420] ERROR in app: Exception on /txtpos [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, 

[0, 0, 0, 0, 0, 0, 1, 1, 1]

[0, 0, 0, 0, 0, 0, 1, 1, 1]

[1, 0, 0, 1, 0, 0, 0, 0, 0]

<ipython-input-6-e857a711d561>:95: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  plt.ylim(0,15)
[2021-02-25 18:18:26,500] ERROR in app: Exception on /txthabit [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", 